In [2]:
import spatialdata
import skimage.io as io
import numpy as np
import opendvp
import pandas as pd

In [ ]:
df = pd.DataFrame(data={"cell_id":[1,2,3], ""}

In [3]:
import pandas as pd

df = pd.DataFrame({
    'color': ['red', 'blue', 'green', 'red']
})

pd.get_dummies(df)


,color_blue,color_green,color_red
0,False,False,True
1,True,False,False
2,False,True,False
3,False,False,True
